<a href="https://colab.research.google.com/github/PeterS111/GPT_2_CODE/blob/main/Open_AI_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open_AI_GPT-2
version 10.06.2022

The code is based on https://github.com/nshepperd/gpt-2 by N Shepperd (which, in turn, is based on the original OpenAI code), with some changes by Peter S.  

This notebook contains the code which was used in our paper "Training GPT-2 to represent two Romantic-era authors challenges, evaluations and pitfalls" to fine-tune and generate text from OpenAI GPT-2 models. 

GPUs on Colab only allow for fine-tuning of the Small (124M) and Medium (345M) models.

Make sure that you are using a GPU: Runtime/Change runtime type/ -> Select "GPU". 

First, check that your runtime is using a GPU:

In [ ]:
# check the GPU:
!nvidia-smi

## 1. Download the repository from github:

In [ ]:
!git clone https://github.com/PeterS111/GPT_2_CODE/

## 2. Select TensorFlow version
Run the following command to ensure you have TensorFlow v 1.x. This code doesn't work with versions 2.x

In [ ]:
%tensorflow_version 1.x

## 3. Change the working directory to the main folder

In [ ]:
cd /content/GPT_2_CODE/Open_AI_GPT_2/

## 4. Install the requirements
After installation you will get the message that "You must restart the runtime in order to use newly installed versions." ⚡**DON'T DO IT!**⚡

In [ ]:
!pip3 install -r requirements.txt 

## 5. Create the 'models' folder

In [ ]:
!mkdir models/ 

## 6. Download the pre-trained GPT-2 model
The command below will download the GPT-2 Small: 124M parameters. If you want GPT-2 Medium version, change the parameter to 345M. Fine-tuning the Large (774M) and XLarge (1558M) is not possible on Colab (for now).

In [ ]:
!python download_model.py 124M

## 7. Batch fine-tune models and generate samples

To run batch fine-tuning and generation go to "run_all.py" and edit accordingly. Uncomment the line below and run it. Please be aware that running that script with its original settings will exceed the maximum running time that Colab allows.

If you want to experiment with a single model and smaller number of samples, please follow form step 8.
 

In [ ]:
#!python run_all.py

### 7.1. Zip the outputs for download 

In [ ]:
#!zip "outputs.zip" outputs/*

## 8. Fine-tuning the model

You can fine tune the models with or without validation dataset. Uncomment as appropriate.
Make sure that "--model_steps" and "--save_every" parameters for "train.py" script and the "--model_steps" for "mover.py" are all the same.

In [ ]:
# Fine-tuning the model without validatation:
!python train.py --dataset "input_data/Shelley.txt" --model_name 124M --max_steps 10000 --save_every 10000
    
# Fine-tuning the model with validatation:
# !python train.py --dataset "input_data/Shelley_train.txt" --val_dataset "input_data/Shelley_val.txt" --val_every 50 --model_name 124M --max_steps 10000 --save_every 10000
 
# This line calls the "mover.py" script to copy the fine-tuned model into the "models" folder
# and to copy from the original pre-trained model the files required to run the fine-tuned model:
!python mover.py --model_steps 10000 --model_size 124M --directory_path "/content/GPT_2_CODE/Open_AI_GPT_2/" 

## 9 (OPTIONAL) Export the fine-tuned model to Google Drive
If you want to save the fine-tuned model for later use, follow these steps:

### 9.1. Zip the model.

In [ ]:
!tar -czvf "my_model_10000_steps.tar.gz" models/model_10000/*

### 9.2. Mount Google Drive (it will require authentication):

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 9.3. Export the compressed model to Drive. This only takes few minutes, but sometimes you may have to run the command twice.

In [ ]:
!cp "my_model_10000_steps.tar.gz" "/content/drive/My Drive/"

## 10 (OPTIONAL) Importing and running the saved model from Google Drive
If you want to generate samples from the saved model (after you have closed this notebook), do the following:

Run step 1. Download the repository from github

Run step 2. Select TensorFlow version

Run step 3. Change the working directory to the main folder

Run step 4. Install the requirements

Run step 5. Create the 'models' folder

After you done that: 



### 10.1. Mount Google Drive, the same as 9.2:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 10.2. Import the previously saved model:

In [ ]:
!cp "/content/drive/My Drive/my_model_10000_steps.tar.gz" "/content/GPT_2_CODE/Open_AI_GPT_2/models/"

### 10.3. Unpack the model and remove the .tar file:

In [ ]:
!tar xf /content/GPT_2_CODE/Open_AI_GPT_2/models/my_model_10000_steps.tar.gz
!rm -v /content/GPT_2_CODE/Open_AI_GPT_2/models/my_model_10000_steps.tar.gz

## 11. Generate samples

Make sure that "--model_name" parameter matches the models you trained in point 8.
For example, if your model was trained for 10000 steps the parameter should be "model_10000".

"seed_start" and "seed_end" variables set the range of samples being generated from your model.

In [ ]:
import os

seed_start = 1
seed_end = 11

prompt = "The eternal sky "
model_descr = "Shelley"
model_name = "model_10000"
length = 1000

prompt = prompt.replace(" ", "£££££")

for s in range(seed_start, seed_end):
    os.system('python generate_conditional_samples_to_file.py --raw_text {prompt} --model_descr {model_descr} --model_name {model_name} --length {length} --seed {s} --temperature 1.0 --top_k 50 --top_p 1.0 --nsamples 1'.format(prompt=prompt, model_descr=model_descr, model_name=model_name, length=length, s=s))
    print("Generating sample with seed: " + str(s))